In [1]:
# # --- Cell 1: Install Libraries (Run Once) ---
# !pip install yfinance alpha_vantage newsapi pandas numpy

In [2]:
!pip uninstall newsapi -y
!pip install newsapi-python

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# --- Cell 2: Import Libraries ---
import yfinance as yf
from alpha_vantage.fundamentaldata import FundamentalData
from newsapi.newsapi_client import NewsApiClient
import pandas as pd
import time
import os

# Create data directories if they don't exist
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)

In [4]:
# import yfinance as yf
# import pandas as pd
# import time

# tickers = ["AAPL", "JPM", "AMZN", "PFE", "XOM"]
# start_date = "2020-01-01"
# end_date = "2023-12-31"

# # Initialize empty DataFrame
# all_prices = pd.DataFrame()

# for ticker in tickers:
#     try:
#         print(f"Downloading {ticker}...")
#         data = yf.download(ticker, start=start_date, end=end_date, progress=False)
#         if not data.empty:
#             all_prices[ticker] = data['Adj Close']
#             print(f"✅ Success: {ticker} ({len(data)} rows)")
#         else:
#             print(f"⚠️ Warning: No data for {ticker}")
#         time.sleep(2)  # Add 2-second delay between requests
#     except Exception as e:
#         print(f"❌ Failed {ticker}: {e}")

# # Save results
# if not all_prices.empty:
#     all_prices.to_csv("../data/raw/stock_prices.csv")
#     print(f"\nSaved data for {len(all_prices.columns)} stocks:\n{all_prices.head()}")
# else:
#     print("\nFailed to download all stocks.")


1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


⚠️ Warning: No data for AAPL



1 Failed download:
['JPM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


⚠️ Warning: No data for JPM



1 Failed download:
['AMZN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


⚠️ Warning: No data for AMZN



1 Failed download:
['PFE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


⚠️ Warning: No data for PFE



1 Failed download:
['XOM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


⚠️ Warning: No data for XOM

Failed to download all stocks.


In [5]:
# --- Cell 4: Fetch Fundamental Data ---
# Replace with your Alpha Vantage API key (free tier: https://www.alphavantage.co/)
av_key = "8M56YMGH1YBO0KYD"

print("Downloading fundamental data...")
fd = FundamentalData(key=av_key, output_format='pandas')

for ticker in tickers:
    try:
        # Get annual balance sheet
        balance_sheet, _ = fd.get_balance_sheet_annual(ticker)
        balance_sheet.to_csv(f"../data/raw/{ticker}_fundamentals.csv")
        print(f"Saved {ticker} fundamentals to ../data/raw/{ticker}_fundamentals.csv")
        time.sleep(12)  # Rate limit (5 requests/minute)
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

Saved AAPL fundamentals to ../data/raw/AAPL_fundamentals.csv
Saved JPM fundamentals to ../data/raw/JPM_fundamentals.csv
Saved AMZN fundamentals to ../data/raw/AMZN_fundamentals.csv
Saved PFE fundamentals to ../data/raw/PFE_fundamentals.csv
Saved XOM fundamentals to ../data/raw/XOM_fundamentals.csv


In [16]:
# --- Cell 5: Fetch News Sentiment Data ---
# Replace with your NewsAPI key (free tier: https://newsapi.org/)
# --- Updated Cell 5: Fetch News Sentiment (Valid Date Range) ---
from newsapi.newsapi_client import NewsApiClient
import pandas as pd
from datetime import datetime, timedelta

# --- Configure Dates ---
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")  # Free tier limit

# --- Fetch News ---
newsapi = NewsApiClient(api_key="ffc2fe4dd7774f61914f971b2a1d77d6")  # Replace with your key

for ticker in ["AAPL", "JPM", "AMZN", "PFE", "XOM"]:
    try:
        articles = newsapi.get_everything(
            q=ticker,
            from_param=start_date,
            to=end_date,
            language="en",
            sort_by="relevancy"
        )
        news_df = pd.DataFrame(articles['articles'])
        if not news_df.empty:
            news_df.to_csv(f"../data/raw/{ticker}_news_recent.csv")
            print(f"✅ Saved {len(news_df)} recent articles for {ticker} ({start_date} to {end_date})")
        else:
            print(f"⚠️ No recent articles found for {ticker}")
    except Exception as e:
        print(f"❌ Failed {ticker}: {str(e)[:100]}...")

✅ Saved 100 recent articles for AAPL (2025-06-24 to 2025-07-24)
✅ Saved 100 recent articles for JPM (2025-06-24 to 2025-07-24)
✅ Saved 100 recent articles for AMZN (2025-06-24 to 2025-07-24)
✅ Saved 33 recent articles for PFE (2025-06-24 to 2025-07-24)
✅ Saved 100 recent articles for XOM (2025-06-24 to 2025-07-24)
